In [1]:
from tensorflow.python.client import device_lib

print('Show system RAM Memory and other details \n\n')
!cat /proc/meminfo | egrep "MemTotal*"

print ('Show devices \n\n ' + str (device_lib.list_local_devices()))

Show system RAM Memory and other details 


MemTotal:       13333580 kB
Show devices 

 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10939668122693685521
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13398612640348319907
physical_device_desc: "device: XLA_CPU device"
]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing libraries and reading data

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as split
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from keras.models import Sequential, Model  
from keras.layers import Dense, LSTM, Dropout, Input
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn import linear_model
from sklearn.externals import joblib
from scipy.optimize import curve_fit
from scipy.stats import normaltest
import pickle

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
%matplotlib inline
sns.set(color_codes=True)
plt.figure(figsize=(25, 15))

<Figure size 1800x1080 with 0 Axes>

<Figure size 1800x1080 with 0 Axes>

In [28]:
df = pd.read_csv('/content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/dataset_00_with_header.csv')
df.head()

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,...,x266,x267,x268,x269,x270,x271,x272,x273,x274,x275,x276,x277,x278,x279,x280,x281,x282,x283,x284,x285,x286,x287,x288,x289,x290,x291,x292,x293,x294,x295,x296,x297,x298,x299,x300,x301,x302,x303,x304,y
0,1540332,NaN,NaN,NaN,8.0,1,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,0,1,0,0,1,2,0,0,0,1,1,0,0,0,1,...,NaN,NaN,NaN,0,0,0,0.0000,0,0,NaN,0,0,0,0,0,0,1,1,1,0,0,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,0,0,0,0,NaN,0,NaN,706
1,823066,4.0,3.0,3.0,4.0,0,2,2,0,0,0,0,0,0,2,2,0,2,2,0,0,0,0,2,0,1,0,0,1,0,14,7,8,10,11,8,1,2,4,5,...,4.0,4.0,0.9256,1,1,1,0.9339,5206,0,NaN,0,0,0,5206,0,5206,1,1,1,2,0,1.0,4.0,3.0,3.5,5206,5206,0.9339,0,NaN,5206,0.9339,1,1,1,0,NaN,0,NaN,558
2,1089795,NaN,NaN,NaN,96.0,1,0,0,0,1,3,4,1,0,9,0,9,2,0,9,0,2,0,0,0,0,1,0,0,3,4,0,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0.2281,1613,1613,0.4814,0,0,0,0,0,0,1,1,1,0,0,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,0,0,0,0,NaN,0,NaN,577
3,1147758,63.0,14.0,38.0,258.0,0,0,0,1,2,1,1,1,1,7,2,5,5,2,4,1,3,1,3,0,0,1,0,0,3,1,0,0,1,1,1,0,0,1,1,...,NaN,NaN,NaN,0,0,0,0.8204,62558,4796,1.1153,1,2,2,58218,58218,0,1,1,1,1,0,2.0,63.0,63.0,63.0,57762,57762,0.8231,57762,0.8231,0,NaN,1,1,1,0,NaN,0,NaN,526
4,1229670,34.0,25.0,29.0,34.0,1,0,0,0,3,0,0,0,0,3,2,1,2,1,3,2,2,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0.1000,190,186,0.1430,1,1,1,190,190,0,1,1,1,0,0,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,0,0,0,0,NaN,0,NaN,496


In [29]:
# Splitting into train and test sets

df_target = df['y']
df = df.drop('y', axis = 1)

train_features, test_features, train_labels, test_labels = train_test_split(df, df_target, test_size=0.25, random_state=42)

# Data Pre-processing

In [30]:
# Removing columns with greater than 7.5% null values

def remove_cols(df):

  columns = df.columns
  for col in columns:
    if(df[col].isnull().sum() >= 7.5*len(df)/100):
      df = df.drop([col], axis = 1)

  df = df.reset_index(drop=True)

  return df

train_features = remove_cols(train_features)
test_features = remove_cols(test_features)

train_features.head()

,x001,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x042,x043,x046,...,x246,x247,x248,x249,x250,x251,x252,x254,x258,x260,x261,x262,x263,x264,x269,x270,x271,x272,x273,x274,x276,x277,x278,x279,x280,x281,x282,x283,x284,x285,x286,x291,x292,x294,x296,x298,x299,x300,x301,x303
0,1280590,347.0,1,0,0,1,0,1,2,0,2,6,3,3,3,2,5,2,2,1,1,0,1,1,0,1,1,3,1,1,2,2,3,1,1,2,2,5000,3884,1,...,1,1,0,0,0,0,0,0,0,0,0,0,1,8256,1,1,1,0.2910,10012,640,0,0,0,9372,0,9372,1,1,1,1,0,8256,8256,0,8256,1,1,1,0,0
1,1164267,115.0,0,0,0,0,0,0,0,0,2,2,0,2,1,0,0,0,0,0,2,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1346906,128.0,1,0,0,0,0,4,4,6,7,21,1,20,4,1,7,1,1,1,12,0,0,1,0,0,7,0,0,0,0,0,0,0,0,0,0,10000,10000,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0576,3952,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0
3,1555326,41.0,0,1,2,3,3,1,0,0,0,9,4,5,4,3,5,2,2,1,4,0,1,1,0,2,5,2,0,0,2,2,2,0,0,2,2,50000,49330,2,...,1,1,0,0,0,0,0,0,0,0,0,0,1,23056,1,1,1,0.2301,42641,0,0,0,0,42641,0,42641,1,1,1,2,0,41971,41971,0,41971,1,1,1,0,18915
4,1366681,35.0,1,0,0,1,1,0,0,0,0,2,0,2,2,0,2,0,2,0,0,0,0,1,0,0,1,3,0,0,0,0,3,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4670,234,234,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0


In [31]:
# Filling Nan values
# Columns with less than 15 distinct values are assumed to be categorical. For these columns, mode will be substituted for NaN values
# Rest of the columns will be assumed to be numerical. For these, mean will be used to replace NaN values.

def fill_nan(df):

  cols = df.columns

  for c in cols:
    if (df[c].nunique() <= 15):
      mode = df[c].mode()[0]
      df[c] = df[c].fillna(mode)
    else:
      mean = round(df[c].mean(),2)
      df[c] = df[c].fillna(mean)

  return df

train_features = fill_nan(train_features)
test_features = fill_nan(test_features)

In [32]:
# Detecting Outliers (Technique 1)
# Finding the number of anomalies and normal points. Here points classified -1 are anomalous

def detect_outlier_1(df):

  to_model_columns = df.columns[:-1]

  clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                    max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

  clf.fit(df[to_model_columns])
  pred = clf.predict(df[to_model_columns])
  df['anomaly'] = pred
  outliers = df.loc[df['anomaly'] == -1]
  outlier_index = list(outliers.index)

  print(len(outlier_index))
  print(df['anomaly'].value_counts())

  return df

train_features = detect_outlier_1(train_features)
test_features = detect_outlier_1(test_features)

9000
 1    66000
-1     9000
Name: anomaly, dtype: int64
3000
 1    22000
-1     3000
Name: anomaly, dtype: int64


In [33]:
# Detecting Outliers (Technique 2)

def detect_outlier_2 (df, df_target):

  df_len = len(df)
  df['y'] = df_target
  cols = df.columns[:-2] 

  Q1 = df[cols].quantile(0.25)
  Q3 = df[cols].quantile(0.75)
  IQR = Q3 - Q1

  #df1 = df[~((df[cols] < (Q1 - 1.5 * IQR)) | (df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
  df1 = df[((df[cols] < (Q1 - 1.5 * IQR)) | (df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
  print ("Number of outliers: ", df_len - len(df1))

  return df1, df_target

#train_features, train_labels = detect_outlier_2(train_features, train_labels)

# Feature Engineering, Selection and Management

In [34]:
# Checking if numerical features are normally distributed. If not, taking their log-normal.

def num_and_cat(df1):
  numericals = []
  categoricals = []

  df1 = df1.drop('anomaly', axis = 1)
  cols = df1.columns

  for c in cols:
    if (df1[c].nunique() <= 15):
      categoricals.append(c)
    else:
      numericals.append(c)

  return numericals, categoricals


def taking_lognormal(df1, numericals):

  alpha = 0.05

  for num in numericals:
    if(any(df1[num]<=0)):
      continue
    else:
      k2, p = normaltest(df1[num].values)
      if (p < alpha):
        df1[num] = df1[num].apply(lambda x: np.log(x))

  return df1

numericals, categoricals = num_and_cat(train_features)
train_features = taking_lognormal(train_features, numericals)
test_features = taking_lognormal(test_features, numericals) 

train_features.head()

,x001,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x042,x043,x046,...,x247,x248,x249,x250,x251,x252,x254,x258,x260,x261,x262,x263,x264,x269,x270,x271,x272,x273,x274,x276,x277,x278,x279,x280,x281,x282,x283,x284,x285,x286,x291,x292,x294,x296,x298,x299,x300,x301,x303,anomaly
0,14.062831,347.0,1,0,0,1,0,1,2,0,2,6,3,3,3,2,5,2,2,1,1,0,1,1,0,1,1,3,1,1,2,2,3,1,1,2,2,5000,3884,1,...,1,0,0,0,0,0,0,0,0,0,0,1,8256,1,1,1,0.2910,10012,640,0,0,0,9372,0,9372,1,1,1,1,0,8256,8256,0,8256,1,1,1,0,0,1
1,13.967602,115.0,0,0,0,0,0,0,0,0,2,2,0,2,1,0,0,0,0,0,2,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,14.113321,128.0,1,0,0,0,0,4,4,6,7,21,1,20,4,1,7,1,1,1,12,0,0,1,0,0,7,0,0,0,0,0,0,0,0,0,0,10000,10000,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0576,3952,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1
3,14.257196,41.0,0,1,2,3,3,1,0,0,0,9,4,5,4,3,5,2,2,1,4,0,1,1,0,2,5,2,0,0,2,2,2,0,0,2,2,50000,49330,2,...,1,0,0,0,0,0,0,0,0,0,0,1,23056,1,1,1,0.2301,42641,0,0,0,0,42641,0,42641,1,1,1,2,0,41971,41971,0,41971,1,1,1,0,18915,1
4,14.127896,35.0,1,0,0,1,1,0,0,0,0,2,0,2,2,0,2,0,2,0,0,0,0,1,0,0,1,3,0,0,0,0,3,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4670,234,234,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1


In [35]:
# Standardizing numerical data

def standardize_numerical(df, numericals):

  for n in numericals:
      df[n] = pd.to_numeric(df[n])
      mean = df[n].mean()
      std = df[n].std()
      df[n] = df[n].apply(lambda x: (x-mean)/std)

  return df 


train_features = standardize_numerical(train_features, numericals)
test_features = standardize_numerical(test_features, numericals)

train_features.head()

,x001,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x042,x043,x046,...,x247,x248,x249,x250,x251,x252,x254,x258,x260,x261,x262,x263,x264,x269,x270,x271,x272,x273,x274,x276,x277,x278,x279,x280,x281,x282,x283,x284,x285,x286,x291,x292,x294,x296,x298,x299,x300,x301,x303,anomaly
0,0.282980,1.400283,1,-0.504969,-0.615871,-0.096333,-0.600668,-0.310815,0.151478,-0.603088,-0.421249,-0.580858,-0.235470,-0.632698,-0.418280,2,-0.229089,-0.089332,2,1,-0.522665,0,1,1,-0.508661,-0.236052,-0.635295,-0.305145,0.835710,0.100038,0.091578,-0.226773,-0.067302,1,1,0.725876,0.059376,-0.225144,-0.195354,0.525778,...,1,0,0,-0.526358,0,0,0,-0.28302,0,0,0,1,0.160534,1,1,1,-0.062370,-0.323692,-0.153154,-0.274639,-0.284907,-0.3094,-0.314181,-0.190082,-0.272126,1,1,1,-0.145563,-0.168816,-0.166231,-0.300912,-0.148534,-0.111916,1,1,1,0,-0.263863,1
1,-0.047816,-0.523399,0,-0.504969,-0.615871,-0.584444,-0.600668,-0.682028,-0.622871,-0.603088,-0.421249,-0.916049,-0.887130,-0.740895,-1.265195,0,-0.867562,-0.723553,0,0,-0.382520,0,1,0,-0.508661,0.253235,-0.844141,-0.691314,-0.301455,-0.445308,-0.546005,-0.620454,-0.929897,0,0,-0.749362,-0.851117,-0.304784,-0.289212,-0.457475,...,0,0,0,-0.526358,0,0,0,-0.28302,0,0,0,0,-0.484595,0,0,0,-0.129521,-0.379440,-0.176464,-0.274639,-0.284907,-0.3094,-0.366573,-0.190082,-0.326635,0,0,0,-0.520764,-0.168816,-0.381749,-0.350462,-0.148534,-0.369095,0,0,0,0,-0.263863,1
2,0.458363,-0.415607,1,-0.504969,-0.615871,-0.584444,-0.600668,0.802826,0.925828,1.679241,0.499367,0.676108,-0.669910,1.206650,0.005177,1,0.026300,-0.406443,1,1,1.018927,0,0,1,-0.508661,-0.725338,0.617785,-0.691314,-0.301455,-0.445308,-0.546005,-0.620454,-0.929897,0,0,-0.749362,-0.851117,-0.145503,-0.047558,-0.457475,...,1,0,0,-0.526358,0,0,0,-0.28302,0,0,0,0,-0.484595,0,0,0,-0.116229,-0.357435,-0.176464,-0.274639,-0.284907,-0.3094,-0.366573,-0.190082,-0.326635,1,1,1,-0.520764,-0.168816,-0.381749,-0.350462,-0.148534,-0.369095,0,0,0,1,-0.263863,1
3,0.958139,-1.136988,0,0.219188,0.270189,0.879888,1.157727,-0.310815,-0.622871,-0.603088,-0.789496,-0.329465,-0.018249,-0.416305,0.005177,3,-0.229089,-0.089332,2,1,-0.102231,0,1,1,-0.508661,0.253235,0.200092,-0.433868,-0.301455,-0.445308,0.091578,-0.226773,-0.354833,0,0,0.725876,0.059376,0.491618,0.902868,1.509032,...,1,0,0,-0.526358,0,0,0,-0.28302,0,0,0,1,1.317014,1,1,1,-0.076423,-0.142010,-0.176464,-0.274639,-0.284907,-0.3094,-0.128195,-0.190082,-0.078629,1,1,1,0.229638,-0.168816,0.713880,-0.098565,-0.148534,0.938328,1,1,1,0,0.516210,1
4,0.508992,-1.186738,1,-0.504969,-0.615871,-0.096333,-0.014536,-0.682028,-0.622871,-0.603088,-0.789496,-0.916049,-0.887130,-0.740895,-0.841738,0,-0.612173,-0.723553,2,0,-0.662810,0,0,1,-0.508661,-0.725338,-0.635295,-0.305145,-0.301455,-0.445308,-0.546005,-0.620454,-0.067302,0,0,-0.749362,-0.851117,-0.304784,-0.289212,-0.457475,...,1,0,0,-0.526358,0,0,0,-0.28302,0,0,0,0,-0.484595,0,0,0,-0.021756,-0.378137,-0.167941,-0.274639,-0.284907,-0.3094,-0.366573,-0.190082,-0.326635,1,1,1,-0.520764,-0.168816,-0.381749,-0.350462,-0.148534,-0.369095,0,0,0,0,-0.263863,1


In [37]:
# Checking if any categorical features contain 'rare' values

numericals, categoricals = num_and_cat(train_features)
rare_percent = 0.01

def check_for_freq(df, col, rare_percent):
  
  tmp = df.groupby(col).count()/len(df)
  freq = tmp[tmp > rare_percent].index
  return freq

def convert_rare(df, categoricals, rare_percent):

  for cat in categoricals:

    mode_value = df[cat].mode()[0]
    freq_ls = check_for_freq(df, cat, rare_percent)
    df[cat] = pd.DataFrame(np.where(df[cat].isin(freq_ls), df[cat], 'Rare'))
    df[cat] = df[cat].replace('Rare', mode_value)

    return df

train_features = convert_rare(train_features, categoricals, rare_percent)
test_features = convert_rare(test_features, categoricals, rare_percent)

In [38]:
# Creating dummies from categorical columns

def create_dummies(df, categoricals):
            
  for c in categoricals:
      dummies = pd.get_dummies(df[c], prefix = c) 
      df = pd.concat([df, dummies], axis=1)
     
  df = df.drop(categoricals, axis = 1) 

  return df 


train_features = create_dummies(train_features, categoricals)
test_features = create_dummies(test_features, categoricals)

print(train_features.shape)
print(test_features.shape)
train_features.head()

(75000, 643)
(25000, 586)


,x001,x005,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x020,x021,x024,x028,x029,x030,x031,x032,x033,x034,x035,x036,x039,x040,x042,x043,x046,x055,x056,x059,x062,x063,x064,x065,x066,x070,...,x254_1,x254_2,x254_3,x254_4,x254_5,x254_6,x254_7,x254_8,x254_9,x254_10,x254_14,x254_21,x260_0,x260_1,x261_0,x261_1,x262_0,x262_1,x263_0,x263_1,x269_0,x269_1,x270_0,x270_1,x271_0,x271_1,x282_0,x282_1,x283_0,x283_1,x284_0,x284_1,x298_0,x298_1,x299_0,x299_1,x300_0,x300_1,x301_0,x301_1
0,0.282980,1.400283,-0.504969,-0.615871,-0.096333,-0.600668,-0.310815,0.151478,-0.603088,-0.421249,-0.580858,-0.235470,-0.632698,-0.418280,-0.229089,-0.089332,-0.522665,-0.508661,-0.236052,-0.635295,-0.305145,0.835710,0.100038,0.091578,-0.226773,-0.067302,0.725876,0.059376,-0.225144,-0.195354,0.525778,-0.366231,-0.407068,-0.496355,-0.347174,-0.424152,-0.490072,-0.493591,-0.286455,-0.188215,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0
1,-0.047816,-0.523399,-0.504969,-0.615871,-0.584444,-0.600668,-0.682028,-0.622871,-0.603088,-0.421249,-0.916049,-0.887130,-0.740895,-1.265195,-0.867562,-0.723553,-0.382520,-0.508661,0.253235,-0.844141,-0.691314,-0.301455,-0.445308,-0.546005,-0.620454,-0.929897,-0.749362,-0.851117,-0.304784,-0.289212,-0.457475,-0.366231,-0.407068,0.532300,-0.347174,0.187947,0.434895,0.190849,-0.286455,-0.188215,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
2,0.458363,-0.415607,-0.504969,-0.615871,-0.584444,-0.600668,0.802826,0.925828,1.679241,0.499367,0.676108,-0.669910,1.206650,0.005177,0.026300,-0.406443,1.018927,-0.508661,-0.725338,0.617785,-0.691314,-0.301455,-0.445308,-0.546005,-0.620454,-0.929897,-0.749362,-0.851117,-0.145503,-0.047558,-0.457475,-0.366231,-0.407068,1.560956,0.689950,0.187947,0.126572,1.787878,-0.286455,-0.188215,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1
3,0.958139,-1.136988,0.219188,0.270189,0.879888,1.157727,-0.310815,-0.622871,-0.603088,-0.789496,-0.329465,-0.018249,-0.416305,0.005177,-0.229089,-0.089332,-0.102231,-0.508661,0.253235,0.200092,-0.433868,-0.301455,-0.445308,0.091578,-0.226773,-0.354833,0.725876,0.059376,0.491618,0.902868,1.509032,-0.366231,-0.407068,-0.496355,-0.347174,-0.424152,-0.490072,-0.493591,-0.286455,-0.188215,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0
4,0.508992,-1.186738,-0.504969,-0.615871,-0.096333,-0.014536,-0.682028,-0.622871,-0.603088,-0.789496,-0.916049,-0.887130,-0.740895,-0.841738,-0.612173,-0.723553,-0.662810,-0.508661,-0.725338,-0.635295,-0.305145,-0.301455,-0.445308,-0.546005,-0.620454,-0.067302,-0.749362,-0.851117,-0.304784,-0.289212,-0.457475,-0.366231,0.368654,-0.290624,-0.347174,-0.424152,-0.181750,-0.265444,-0.286455,-0.188215,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,1,0


In [39]:
# Selecting the common columns from both the train and test set

common_cols = train_features.columns.intersection(test_features.columns)

train_features = train_features[common_cols]
test_features = test_features[common_cols]

print(train_features.shape)
print(test_features.shape)
train_features.head()

(75000, 563)
(25000, 563)


,x001,x005,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x020,x021,x024,x028,x029,x030,x031,x032,x033,x034,x035,x036,x039,x040,x042,x043,x046,x055,x056,x059,x062,x063,x064,x065,x066,x070,...,x252_1,x252_2,x252_3,x254_0,x254_1,x254_2,x254_3,x254_4,x254_5,x254_6,x254_8,x254_10,x260_0,x260_1,x261_0,x261_1,x262_0,x262_1,x263_0,x263_1,x269_0,x269_1,x270_0,x270_1,x271_0,x271_1,x282_0,x282_1,x283_0,x283_1,x284_0,x284_1,x298_0,x298_1,x299_0,x299_1,x300_0,x300_1,x301_0,x301_1
0,0.282980,1.400283,-0.504969,-0.615871,-0.096333,-0.600668,-0.310815,0.151478,-0.603088,-0.421249,-0.580858,-0.235470,-0.632698,-0.418280,-0.229089,-0.089332,-0.522665,-0.508661,-0.236052,-0.635295,-0.305145,0.835710,0.100038,0.091578,-0.226773,-0.067302,0.725876,0.059376,-0.225144,-0.195354,0.525778,-0.366231,-0.407068,-0.496355,-0.347174,-0.424152,-0.490072,-0.493591,-0.286455,-0.188215,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0
1,-0.047816,-0.523399,-0.504969,-0.615871,-0.584444,-0.600668,-0.682028,-0.622871,-0.603088,-0.421249,-0.916049,-0.887130,-0.740895,-1.265195,-0.867562,-0.723553,-0.382520,-0.508661,0.253235,-0.844141,-0.691314,-0.301455,-0.445308,-0.546005,-0.620454,-0.929897,-0.749362,-0.851117,-0.304784,-0.289212,-0.457475,-0.366231,-0.407068,0.532300,-0.347174,0.187947,0.434895,0.190849,-0.286455,-0.188215,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
2,0.458363,-0.415607,-0.504969,-0.615871,-0.584444,-0.600668,0.802826,0.925828,1.679241,0.499367,0.676108,-0.669910,1.206650,0.005177,0.026300,-0.406443,1.018927,-0.508661,-0.725338,0.617785,-0.691314,-0.301455,-0.445308,-0.546005,-0.620454,-0.929897,-0.749362,-0.851117,-0.145503,-0.047558,-0.457475,-0.366231,-0.407068,1.560956,0.689950,0.187947,0.126572,1.787878,-0.286455,-0.188215,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1
3,0.958139,-1.136988,0.219188,0.270189,0.879888,1.157727,-0.310815,-0.622871,-0.603088,-0.789496,-0.329465,-0.018249,-0.416305,0.005177,-0.229089,-0.089332,-0.102231,-0.508661,0.253235,0.200092,-0.433868,-0.301455,-0.445308,0.091578,-0.226773,-0.354833,0.725876,0.059376,0.491618,0.902868,1.509032,-0.366231,-0.407068,-0.496355,-0.347174,-0.424152,-0.490072,-0.493591,-0.286455,-0.188215,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0
4,0.508992,-1.186738,-0.504969,-0.615871,-0.096333,-0.014536,-0.682028,-0.622871,-0.603088,-0.789496,-0.916049,-0.887130,-0.740895,-0.841738,-0.612173,-0.723553,-0.662810,-0.508661,-0.725338,-0.635295,-0.305145,-0.301455,-0.445308,-0.546005,-0.620454,-0.067302,-0.749362,-0.851117,-0.304784,-0.289212,-0.457475,-0.366231,0.368654,-0.290624,-0.347174,-0.424152,-0.181750,-0.265444,-0.286455,-0.188215,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,1,0


In [40]:
# Feature Selection (Technique 1)

def feature_selection_1(df1, df_target):

  model = SelectKBest(f_regression, k=375).fit(df1, df_target)
  list_form = list(model.scores_)

  res = sorted(range(len(list_form)), key = lambda sub: list_form[sub])[-375:]
  temp = df1.iloc[:, res]
  sel_1_cols = temp.columns
  
  print(sel_1_cols)

  return temp, sel_1_cols

df2, sel_1_cols = feature_selection_1(train_features, train_labels)

Index(['x122_7', 'x163_5', 'x251_6', 'x175_5', 'x228_8', 'x100_6', 'x053_0',
       'x182_2', 'x176_6', 'x101_5',
       ...
       'x244_0', 'x046', 'x229_0', 'x226', 'x249_1', 'x249_0', 'x227', 'x005',
       'x236', 'x228_0'],
      dtype='object', length=375)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [21]:
df2.head()

,x069,x033,x011,x012,x038,x001,x049,x070,x074,x061,x013,x071,x076,x048,x034,x016,x072,x066,x026,x039,x029,x021,x075,x035,x031,x073,x062,x040,x017,x047,x042,x015,x036,x056,x043,x063,x064,x065,x059,x018,...,x117,x194,x112,x147,x187,x240,x104,x281,x284,x193,x192,x171,x283,x282,x099,x276,x172,x277,anomaly,x278,x173,x168,x251,x248,x224,x260,x261,x229,x247,x262,x250,x246,x245,x225,x244,x228,x226,x249,x227,x236
0,0,0.100038,-0.310815,0.151478,1,0.282980,1,-0.188215,-0.240104,0,-0.603088,-0.249249,-0.343906,0,0.091578,-0.235470,-0.275271,-0.286455,1,0.725876,-0.236052,-0.089332,-0.306308,-0.226773,-0.305145,-0.335270,-0.347174,0.059376,-0.632698,1,-0.225144,-0.580858,-0.067302,-0.407068,-0.195354,-0.424152,-0.490072,-0.493591,-0.496355,-0.418280,...,-0.230804,-0.314175,-0.265673,0,-0.27027,-0.082520,-0.211344,-0.272126,1,-0.284294,-0.258232,-0.327852,1,1,-0.274639,-0.274639,-0.378718,-0.284907,1,-0.3094,-0.441773,0.136875,0,0,0.221413,0,0,0,1,0,-0.526358,1,1,-0.094035,1,0,0.006030,0,-0.530784,-0.281835
1,0,-0.445308,-0.682028,-0.622871,0,-0.047816,0,-0.188215,-0.240104,0,-0.603088,-0.249249,-0.343906,0,-0.546005,-0.887130,-0.275271,-0.286455,1,-0.749362,0.253235,-0.723553,0.893542,-0.620454,-0.691314,0.852765,-0.347174,-0.851117,-0.740895,0,-0.304784,-0.916049,-0.929897,-0.407068,-0.289212,0.187947,0.434895,0.190849,0.532300,-1.265195,...,-0.230804,-0.314175,-0.265673,0,-0.27027,-0.534667,-0.211344,-0.326635,0,-0.284294,-0.258232,-0.327852,0,0,-0.274639,-0.274639,-0.378718,-0.284907,1,-0.3094,-0.441773,0.136875,0,0,-0.637598,0,0,0,0,0,-0.526358,0,0,-0.605580,0,0,-0.576019,0,-0.530784,-0.482470
2,0,-0.445308,0.802826,0.925828,0,0.458363,0,-0.188215,-0.240104,0,1.679241,-0.249249,-0.343906,0,-0.546005,-0.669910,-0.275271,-0.286455,0,-0.749362,-0.725338,-0.406443,1.434058,-0.620454,-0.691314,1.565585,0.689950,-0.851117,1.206650,0,-0.145503,0.676108,-0.929897,-0.407068,-0.047558,0.187947,0.126572,1.787878,1.560956,0.005177,...,-0.230804,0.568764,-0.265673,0,-0.27027,-0.082520,-0.211344,-0.326635,1,-0.284294,-0.258232,-0.327852,1,1,-0.274639,-0.274639,-0.378718,-0.284907,1,-0.3094,-0.441773,-0.478683,0,0,-0.208093,0,0,0,1,0,-0.526358,1,1,-0.094035,1,0,0.006030,0,0.176136,0.034098
3,0,-0.445308,-0.310815,-0.622871,0,0.958139,0,-0.188215,-0.240104,0,-0.603088,-0.249249,-0.343906,0,0.091578,-0.018249,-0.275271,-0.286455,1,0.725876,0.253235,-0.089332,-0.306308,-0.226773,-0.433868,-0.335270,-0.347174,0.059376,-0.416305,0,0.491618,-0.329465,-0.354833,-0.407068,0.902868,-0.424152,-0.490072,-0.493591,-0.496355,0.005177,...,-0.230804,-0.314175,-0.265673,0,-0.27027,-0.534667,-0.211344,-0.078629,1,-0.284294,-0.258232,-0.327852,1,1,-0.274639,-0.274639,-0.378718,-0.284907,1,-0.3094,-0.441773,-0.478683,0,0,0.221413,0,0,2,1,0,-0.526358,1,1,0.417509,1,2,0.588079,0,0.883057,2.065761
4,0,-0.445308,-0.682028,-0.622871,0,0.508992,0,-0.188215,-0.240104,0,-0.603088,-0.249249,-0.343906,0,-0.546005,-0.887130,-0.275271,-0.286455,0,-0.749362,-0.725338,-0.723553,-0.226843,-0.620454,-0.305145,-0.097663,-0.347174,-0.851117,-0.740895,0,-0.304784,-0.916049,-0.067302,0.368654,-0.289212,-0.424152,-0.181750,-0.265444,-0.290624,-0.841738,...,-0.230804,-0.314175,-0.265673,0,-0.27027,-0.082520,-0.211344,-0.326635,1,-0.284294,-0.258232,0.599716,1,1,-0.274639,-0.274639,0.420400,-0.284907,1,-0.3094,0.231087,0.136875,0,0,-0.637598,0,0,0,1,0,-0.526358,1,1,-0.605580,0,0,-0.576019,0,-0.530784,-0.482470


In [41]:
# Feature Selection (Technique 2)

def feature_selection_2(df1, df_target):

  model = DecisionTreeRegressor(max_depth = 10, min_samples_leaf = 5000)
  rfe = RFE(model,375).fit(df1, df_target)

  sel_2_cols = []

  for index, value in enumerate(rfe.support_):
    if (value):
      sel_2_cols.append(df1.columns[index])

  print(sel_2_cols)

  return sel_2_cols

sel_2_cols = feature_selection_2(train_features, train_labels)

['x005', 'x007', 'x008', 'x009', 'x010', 'x011', 'x012', 'x013', 'x014', 'x015', 'x016', 'x017', 'x018', 'x020', 'x021', 'x024', 'x028', 'x029', 'x030', 'x031', 'x032', 'x033', 'x034', 'x035', 'x036', 'x039', 'x040', 'x042', 'x043', 'x046', 'x055', 'x056', 'x059', 'x062', 'x063', 'x064', 'x065', 'x066', 'x070', 'x071', 'x072', 'x073', 'x074', 'x075', 'x076', 'x081', 'x097', 'x099', 'x102', 'x103', 'x104', 'x105', 'x106', 'x109', 'x110', 'x111', 'x112', 'x113', 'x114', 'x115', 'x116', 'x117', 'x118', 'x119', 'x120', 'x121', 'x123', 'x124', 'x125', 'x126', 'x127', 'x128', 'x129', 'x130', 'x131', 'x132', 'x133', 'x134', 'x135', 'x136', 'x137', 'x138', 'x139', 'x140', 'x141', 'x142', 'x143', 'x144', 'x145', 'x146', 'x149', 'x150', 'x151', 'x152', 'x153', 'x157', 'x158', 'x159', 'x160', 'x164', 'x165', 'x166', 'x167', 'x168', 'x169', 'x170', 'x171', 'x172', 'x173', 'x174', 'x178', 'x179', 'x181', 'x183', 'x184', 'x185', 'x186', 'x187', 'x188', 'x189', 'x190', 'x191', 'x192', 'x193', 'x194',

In [42]:
# Feature Selection (Technique 3)

def feature_selection_3(df1, df_target):

  model = RandomForestRegressor(max_depth = 10, min_samples_leaf = 5000)
  model.fit(df1, df_target)

  feature_list = list(model.feature_importances_)
  result = sorted(range(len(feature_list)), key = lambda sub: feature_list[sub])[-375:]
  df3 = df1.iloc[:, result]
  sel_3_cols = df3.columns

  return sel_3_cols

sel_3_cols = feature_selection_3(train_features, train_labels)

In [43]:
# Taking the features which are common to all 3 techniques

final_col_list = list(set(sel_1_cols) & set(sel_2_cols) & set(sel_3_cols))

train_features = train_features[final_col_list]
test_features = test_features[final_col_list]

In [44]:
print(train_features.shape)
print(test_features.shape)
train_features.head()

(75000, 140)
(25000, 140)


,x251_0,x122_6,x249_1,x233,x156_1,x269_0,x107_2,x263_0,x272,x182_1,x175_4,x181,x252_0,x122_4,x300_0,x248_0,x228_5,x186,x228_7,x262_0,x228_1,x273,x122_5,x177_1,x176_2,x156_2,x251_3,x229_3,x180_0,x300_1,x175_3,x163_0,x229_1,x251_1,x301_0,x101_4,x245_1,x284_1,x175_0,x108_1,...,x298_1,x245_0,x228_0,x122_2,x248_1,x177_3,x176_3,x161_0,x271_1,x147_0,x176_4,x229_5,x270_1,x176_6,x163_2,x177_6,x107_3,x252_1,x269_1,x177_5,x240,x261_1,x262_1,x270_0,x261_0,x005,x301_1,x284_0,x108_4,x299_1,x182_0,x154_0,x156_0,x043,x229_4,x175_5,x108_0,x274,x163_3,x244_0
0,1,0,0,-0.274750,0,0,0,0,-0.062370,0,0,-0.510244,1,0,0,1,0,-0.393986,0,1,0,-0.323692,0,0,0,0,0,0,1,1,0,1,0,0,1,0,1,1,1,0,...,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,-0.082520,0,0,0,1,1.400283,0,0,0,1,1,1,1,-0.195354,0,0,1,-0.153154,0,0
1,1,0,0,-0.438527,0,1,0,1,-0.129521,0,0,-0.510244,1,0,1,1,0,-0.393986,0,1,0,-0.379440,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,...,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-0.534667,0,0,1,1,-0.523399,0,1,0,0,1,1,1,-0.289212,0,0,1,-0.176464,0,1
2,1,0,0,-0.438527,0,1,0,1,-0.116229,0,0,2.510349,1,0,1,1,0,5.926835,0,1,0,-0.357435,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,1,0,...,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-0.082520,0,0,1,1,-0.415607,1,0,0,0,1,1,1,-0.047558,0,0,1,-0.176464,0,0
3,1,0,0,-0.376038,0,0,0,0,-0.076423,0,0,-0.510244,1,0,0,1,0,-0.393986,0,1,0,-0.142010,0,0,0,0,0,0,1,1,0,1,0,0,1,0,1,1,1,0,...,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,-0.534667,0,0,0,1,-1.136988,0,0,0,1,1,1,1,0.902868,0,0,1,-0.176464,0,0
4,1,0,0,-0.416702,0,1,0,1,-0.021756,0,0,-0.510244,1,0,1,1,0,-0.393986,0,1,0,-0.378137,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,...,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-0.082520,0,0,1,1,-1.186738,0,0,0,0,1,1,1,-0.289212,0,0,1,-0.167941,0,1


In [45]:
train_features.to_csv('/content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/train_features.csv')
test_features.to_csv('/content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/test_features.csv')

# Model Development and Evaluation

In [48]:
# Building models and evaluating their performance

def model_making1(train_features, train_labels, algo, param_grid, name):
    
    model = GridSearchCV(estimator = algo, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 1)
    model.fit(train_features, train_labels)
    print ('Best parameters of', name, 'model: ', model.best_params_)
    best_model = model.best_estimator_
    
    return best_model


def test_for_3(x):

  x = int(x)

  if (x <= 3):
    return 1
  else:
    return 0


def performance_eval(model, test_features, test_labels, name):
    
    prediction = model.predict(test_features)
    
    df = pd.DataFrame()
    df['Actual'] = test_labels.values
    df['Prediction'] = prediction
    df['Pred_Error'] = abs(100*(df['Prediction']- df['Actual'])/df['Actual'])
    mape = df['Pred_Error'].mean()

    df['Diff'] = abs(df['Actual'] - df['Prediction'])
    df['Accuracy'] = df['Diff'].apply(lambda x: test_for_3(x))
    accuracy = 100*df['Accuracy'].mean()

    rmse = math.sqrt(mean_squared_error(test_labels.values, prediction))

    r2 = r2_score(test_labels.values, prediction)
    
    print ('Model Name:', name, ', RMSE: ', rmse)
    print ('Model Name:', name, ', Accuracy (%): ', accuracy)
    print ('Model Name:', name, ', MAPE(%): ', mape)
    print ('Model Name:', name, ', R2 Score: ', r2)

    return prediction

In [49]:
# Building a Lasso model

lasso = linear_model.Lasso()
param_grid_lasso = {'alpha':[1, .5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0]}

model_lasso_v2 = model_making1(train_features, train_labels, lasso, param_grid_lasso, 'Lasso')
pred_lasso = performance_eval(model_lasso_v2, test_features, test_labels, 'Lasso')

Model Name: Lasso , RMSE:  59.302631913296274
Model Name: Lasso , Accuracy (%):  5.648000000000001
Model Name: Lasso , MAPE(%):  7.7923533403305445
Model Name: Lasso , R2 Score:  0.7505417977474678


In [50]:
# Building a DNN model

training_set_size = train_features.shape[1]

model_nn2 = Sequential()
model_nn2.add(Dense(100, input_dim=training_set_size, kernel_initializer='normal', activation='relu'))
model_nn2.add(Dense(300, kernel_initializer='normal', activation='relu'))
model_nn2.add(Dense(150, kernel_initializer='normal', activation='relu'))
model_nn2.add(Dense(300, kernel_initializer='normal', activation='relu'))
model_nn2.add(Dense(150, kernel_initializer='normal', activation='relu'))
model_nn2.add(Dense(1, kernel_initializer='normal'))
model_nn2.compile(loss='mean_squared_error', optimizer='adam')
model_nn2.fit(train_features, train_labels, epochs= 15, batch_size=50, verbose=2)

pred_dnn = performance_eval(model_nn2, test_features, test_labels, 'NN-6Layers')

Epoch 1/15
1500/1500 - 5s - loss: 9167.3809
Epoch 2/15
1500/1500 - 5s - loss: 2074.1375
Epoch 3/15
1500/1500 - 5s - loss: 1970.2008
Epoch 4/15
1500/1500 - 5s - loss: 1891.8457
Epoch 5/15
1500/1500 - 5s - loss: 1850.5016
Epoch 6/15
1500/1500 - 5s - loss: 1824.7296
Epoch 7/15
1500/1500 - 5s - loss: 1789.0740
Epoch 8/15
1500/1500 - 5s - loss: 1743.6577
Epoch 9/15
1500/1500 - 5s - loss: 1712.2937
Epoch 10/15
1500/1500 - 5s - loss: 1678.3047
Epoch 11/15
1500/1500 - 5s - loss: 1692.9904
Epoch 12/15
1500/1500 - 5s - loss: 1645.9613
Epoch 13/15
1500/1500 - 5s - loss: 1653.4160
Epoch 14/15
1500/1500 - 6s - loss: 1615.7070
Epoch 15/15
1500/1500 - 5s - loss: 1638.7046
Model Name: NN-6Layers , RMSE:  46.86430537325628
Model Name: NN-6Layers , Accuracy (%):  6.468
Model Name: NN-6Layers , MAPE(%):  6.2052412033081055
Model Name: NN-6Layers , R2 Score:  0.8442119216652683


In [51]:
# Building a XGBoost model

xgb = XGBRegressor() 

param_grid_xgb = {'colsample_bytree': [0.2, 0.4, 0.6],
                  'gamma': [0],
                  'learning_rate': [0.05],
                  'max_depth': [6],
                  'min_child_weight':[1.5],
                  'n_estimator':[3000, 5000, 7200],
                  'reg_alpha':[0.9],
                  'reg_lambda':[0.6],
                  'subsample':[0.2],
                  'seed':[42],
                  'silent':[1],
                  'random_state':[7]
                  }

model_xgb_v2 = model_making1(train_features, train_labels, xgb, param_grid_xgb, 'XGBoost')
pred_xgb = performance_eval(model_xgb_v2, test_features, test_labels, 'XGBoost')

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  5.2min finished


Best parameters of XGBoost model:  {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1.5, 'n_estimator': 3000, 'random_state': 7, 'reg_alpha': 0.9, 'reg_lambda': 0.6, 'seed': 42, 'silent': 1, 'subsample': 0.2}
Model Name: XGBoost , RMSE:  73.45031146224915
Model Name: XGBoost , Accuracy (%):  3.496
Model Name: XGBoost , MAPE(%):  9.421148300170898
Model Name: XGBoost , R2 Score:  0.6173187928817505


In [54]:
from joblib import dump, load

dump(model_lasso_v2, '/content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/lasso.joblib')
model_nn2.save('/content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/')
dump(model_xgb_v2, '/content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/xgb.joblib')

INFO:tensorflow:Assets written to: /content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/assets


['/content/drive/My Drive/Job_Test/WalletHub/Data_Scientist_Test/Data/xgb.joblib']

# Ensembling

In [56]:
# Checking if ensembling improves results

result = pd.DataFrame()
result['Lasso'] = pred_lasso
result['DNN'] = pred_dnn
result['XGB'] = pred_xgb
result['Actual'] = test_labels.values
result['Ensem_1'] = round((result['Lasso']+result['DNN']+result['XGB'])/3, 2)
result.head()

,Lasso,DNN,XGB,Actual,Ensem_1
0,560.406629,583.017273,534.485107,625,559.30
1,477.765236,401.664703,405.718414,501,428.38
2,523.117132,630.964050,512.405762,660,555.50
3,541.912904,530.085571,465.356628,512,512.45
4,573.469984,580.721436,446.649994,527,533.61


In [60]:
def performance_eval_ensem(df, col_name):
    
  df['Prediction'] = df[col_name]
  prediction = df['Prediction'].values
  test_labels = df['Actual'].values

  df['Pred_Error'] = abs(100*(df['Prediction']- df['Actual'])/df['Actual'])
  mape = df['Pred_Error'].mean()

  df['Diff'] = abs(df['Actual'] - df['Prediction'])
  df['Accuracy'] = df['Diff'].apply(lambda x: test_for_3(x))
  accuracy = 100*df['Accuracy'].mean()

  rmse = math.sqrt(mean_squared_error(test_labels, prediction))

  r2 = r2_score(test_labels, prediction)
    
  print ('RMSE: ', rmse)
  print ('Accuracy (%): ', accuracy)
  print ('MAPE(%): ', mape)
  print ('R2 Score: ', r2)

  return 0

In [61]:
performance_eval_ensem(result, 'Ensem_1')

RMSE:  44.87455780354833
Accuracy (%):  8.672
MAPE(%):  5.695146222803431
R2 Score:  0.857159876885495


0

In [62]:
result['Ensem_2'] = round((result['Lasso']+result['DNN'])/2, 2)
performance_eval_ensem(result, 'Ensem_2')

RMSE:  45.581574503345095
Accuracy (%):  7.643999999999999
MAPE(%):  6.064332466273448
R2 Score:  0.8526234121612611


0